In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as  F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType , StructField , StringType , LongType , DoubleType , IntegerType

In [2]:
spark = SparkSession.builder.appName("Football Streaming realtime") \
                            .config("spark.streaming.stopGracefullyOnShutdown" , True) \
                            .config("spark.jars.packages" , "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.1") \
                            .config("spark.jars" , '/home/mhai/Project DE/EXAM_DATA/Week4/postgresql-42.7.3.jar') \
                            .config("spark.sql.shuffle.partitions" , 5) \
                            .master("local[*]") \
                            .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/02 12:43:33 WARN Utils: Your hostname, Admin-PC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/02 12:43:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/mhai/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/mhai/.ivy2.5.2/cache
The jars for the packages stored in: /home/mhai/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6078bbf9-3e64-4f4c-9871-6507e331268f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.s

In [3]:
op_schema = StructType([
        StructField('Unnamed: 0' , LongType() , True) ,
        StructField('average_rating' , DoubleType() , True) ,
        StructField('episode' , LongType() , True) ,
        StructField('name' , StringType() , True) ,
        StructField('rank' , StringType() , True) ,
        StructField('season' , LongType() , True) ,
        StructField('start' , LongType() ,True) ,
        StructField('total_votes' , StringType() , True) ,
        StructField('trend' , StringType() , True)
    ])

In [4]:
op_kafka = spark.readStream.format('kafka') \
                            .option('kafka.bootstrap.servers' , '172.23.152.231:9092') \
                            .option('subscribe' , 'one-piece') \
                            .option('startingOffsets' , 'earliest')\
                            .load()

In [5]:
def generate_op(op_kafka) :
    one_piece = op_kafka.withColumn('value' , col('value').cast('string'))
    one_piece = one_piece.withColumn('value_json' , from_json(col('value') , schema = op_schema)).select('value_json.*')
    # Đổi tên cột
    one_piece = one_piece.withColumnRenamed('Unnamed: 0' , 'op_id').withColumnRenamed('start' , 'year')
    # Chuyển đổi kiểu dữ liệu
    one_piece = one_piece.withColumn('rank' , regexp_replace('rank' , ',' , '').cast('long') ) \
                            .withColumn('total_votes' , regexp_replace('total_votes' , ',' , '').cast('long')  ) \
                                .withColumn('trend' , regexp_replace('trend' , '-' , '0'))
    return one_piece

In [6]:
def op_per_year(one_piece):
    op_per_year = one_piece.groupBy('year').agg(F.avg('average_rating').alias('avg_rating') 
                                , F.sum('total_votes').alias('total_votes')) \
                                    .orderBy('year')
    return op_per_year

In [12]:
def op_rating(one_piece) :
    op_rating = one_piece.withColumn('rating' , when(col('average_rating') >= 9.0 , 'Excellent') \
                                        .when(col('average_rating') >= 7.0 , 'Great') \
                                        .when(col('average_rating') >= 5.0 , 'Average') \
                                        .when(col('average_rating') >= 3.0 , 'Poor') \
                                        .otherwise('Terrible'))
    op_rating = op_rating.select('episode' , 'name' , 'year' , 'trend'  , 'rating')
    return op_rating
                                

In [13]:
def save_psql(df  , batch_id) : 
    print("=== LOAD TABLE ===")
    print('batch id : ' , str(batch_id))

    opiece_per_year = op_per_year(df)
    opiece_rating = op_rating(df)
    
    print("===== Save op_per_year into PostgreSQL =====")
    opiece_per_year.write.format('jdbc') \
            .mode('append') \
            .option('driver' , 'org.postgresql.Driver' ) \
            .option('url' , 'jdbc:postgresql://192.168.1.3:5432/one_piece') \
            .option('user' , 'postgres') \
            .option('password' , 'MINHHAI123') \
            .option('dbtable' , 'public.op_per_year') \
            .save()
    
    print("===== Save op_rating into PostgreSQL =====")
    opiece_rating.write.format('jdbc') \
            .mode('append') \
            .option('driver' , 'org.postgresql.Driver' ) \
            .option('url' , 'jdbc:postgresql://192.168.1.3:5432/one_piece') \
            .option('user' , 'postgres') \
            .option('password' , 'MINHHAI123') \
            .option('dbtable' , 'public.op_rating') \
            .save()
    print('=== Save into postgreSQL Successfully ===')

In [14]:
def writestream_psql(df) :
    print('='*90)
    print('=== Start the process of writing data to postgreSql ===')
    print('='*90)

    df.writeStream.outputMode('append') \
                    .foreachBatch(save_psql) \
                    .trigger(processingTime = '15 seconds') \
                    .option('checkpointLocation' , '../check_dir') \
                    .start() \
                    .awaitTermination()
    

In [16]:
if __name__ == "__main__" :
    print("=== Load Kafka ===")
    one_piece = generate_op(op_kafka)

    writestream_psql(one_piece)
    

    

=== Load Kafka ===
=== Start the process of writing data to postgreSql ===


25/11/02 13:10:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/11/02 13:10:20 WARN StreamingQueryManager: Stopping existing streaming query [id=8e3de9ab-670b-4f96-a1ed-66c286863c9c, runId=343be31e-228f-4187-a2a1-c3b78c0166f6], as a new run is being started.
25/11/02 13:10:20 WARN DAGScheduler: Failed to cancel job group 343be31e-228f-4187-a2a1-c3b78c0166f6. Cannot find active jobs for it.
25/11/02 13:10:20 WARN DAGScheduler: Failed to cancel job group 343be31e-228f-4187-a2a1-c3b78c0166f6. Cannot find active jobs for it.


=== LOAD TABLE ===
batch id :  381
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  382
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  383
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  384
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====


=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  385
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====


=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  386
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====


...

=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  990
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  991
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  992
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  993
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====


=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  994
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  995
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===
=== LOAD TABLE ===
batch id :  996
===== Save op_per_year into PostgreSQL =====
===== Save op_rating into PostgreSQL =====
=== Save into postgreSQL Successfully ===


25/11/02 15:44:51 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:51 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:52 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:52 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:53 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:54 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be establis

KeyboardInterrupt: 

25/11/02 15:44:58 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:44:59 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:45:00 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:45:01 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:45:02 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be established. Node may not be available.
25/11/02 15:45:03 WARN NetworkClient: [AdminClient clientId=adminclient-6] Connection to node 1 (/172.23.152.231:9092) could not be establis